# H3 Python API

In [58]:
from h3 import h3
import folium
from json import load

def aggregated_hex_points(points: list) -> dict:
    """
    points is a list of points. Each point is a list of lat and lng.
    eg. [[lat1, lng1], [lat2, lng2]]
    """
    hexagons = {}
    for point in points:
        hex = h3.geo_to_h3(point[0], point[1], 4)
        if hex not in hexagons:
            hexagons[hex] = 0
        hexagons[hex] += 1
    return hexagons

def style(feature):
        return {
            'fillColor': feature['properties']['color'],
            'color': feature['properties']['color'],
            'weight': 1
        }

with open("poland.country.json") as f:
    borders = load(f)

with open('aed_poland.geojson') as ps:
    pointsR = load(ps)

geoJson_border = borders["features"][0]["geometry"]
# swap lat and lng
geoJson_border["coordinates"][0] = [[p[1], p[0]] for p in geoJson_border["coordinates"][0]]

polyline = geoJson_border["coordinates"][0]


In [59]:
lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]

m = folium.Map(
    location=[sum(lat) / len(lat), sum(lng) / len(lng)],
    zoom_start=5,
    tiles="cartodbpositron",
)

border_polyline = folium.PolyLine(locations=polyline, weight=8, color="green")
m.add_child(border_polyline)

points_coordinates = [feature['geometry']['coordinates'] for feature in pointsR['features']]
points_coordinates = [[p[1], p[0]] for p in points_coordinates]

h3_aggregated = aggregated_hex_points(points_coordinates)

h3_hexes = set(h3_aggregated.keys())
hexagons = list(h3_hexes)
polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v: v[0], polyline))
    lng.extend(map(lambda v: v[1], polyline))
    polylines.append(polyline)
for polyline in polylines:
    border_polyline = folium.PolyLine(locations=polyline, weight=8, style_function=style)
    m.add_child(border_polyline)
display(m)

In [60]:
lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]

m = folium.Map(
    location=[sum(lat) / len(lat), sum(lng) / len(lng)],
    zoom_start=5,
    tiles="cartodbpositron",
)

border_polyline = folium.PolyLine(locations=polyline, weight=8, color="green")
m.add_child(border_polyline)

points_coordinates = [feature['geometry']['coordinates'] for feature in pointsR['features']]
points_coordinates = [[p[1], p[0]] for p in points_coordinates]

h3_aggregated = aggregated_hex_points(points_coordinates)

h3_hexes = set(h3_aggregated.keys())
hexagons = list(h3_hexes)
polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v: v[0], polyline))
    lng.extend(map(lambda v: v[1], polyline))
    polylines.append(polyline)
for polyline in polylines:
    border_polyline = folium.PolyLine(locations=polyline, weight=8, style_function=style)
    m.add_child(border_polyline)
display(m)